In [1]:
import matplotlib as plt
import pandas as pd
import sklearn as sk
import seaborn as sns

In [ ]:
#import imblearn
#import collections

In [2]:
from pandas import read_csv
from sklearn.metrics import fbeta_score,make_scorer
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import RepeatedStratifiedKFold


In [15]:

#from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

In [3]:
df = pd.read_csv("df_clean2.csv",delimiter=',')

In [4]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [5]:
def prepare_dataset(df):
    last_col = len(df.columns) - 1
    X, y = df.iloc[:,0:last_col-1],df.iloc[:,last_col] 
    return X, y

In [6]:
def f2_score(y_true, y_pred):
    return fbeta_score(y_true, y_pred, beta=2)

In [7]:
# model evaluation
def evaluate_model(X, y, model):
    # evaluation procedure
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    #evaluation metric
    metric = make_scorer(f2_score)
    f2_scores = cross_val_score(model, X, y, scoring=metric, cv=cv, n_jobs=-1)
    accuracy_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1) 
    return f2_scores,accuracy_scores

In [8]:
df.sample(5)

,transaction_amount,is_declined,total_number_declines_per_day,is_foreign_transaction,is_high_risk_country,daily_chargeback_avg_amt,6_month_avg_chbk_amt,6-month_chbk_freq,is_fraudulent
559,800.912478,0,0,0,0,0,0.0,0,0
797,6112.294969,0,0,1,0,0,0.0,0,0
1059,10879.002710,0,0,0,0,0,0.0,0,0
2809,6321.603145,0,0,0,0,0,0.0,0,0
1450,2939.882165,0,0,0,0,0,0.0,0,0


## Let's make a baseline out of a naive classifier

In [9]:
# Separating features and target

In [10]:
X, y = prepare_dataset(df)

In [13]:
#Creating a dummy classifier that always predict the minority class as a baseline
model = DummyClassifier(strategy='constant', constant=1)

In [16]:
# Evaluate the dummy classifier
f2_scores, accuracy_scores = evaluate_model(X, y, model)

In [17]:
print('Mean accuracy: %.3f (%.3f)' % (accuracy_scores.mean(), accuracy_scores.std()))

Mean accuracy: 0.150 (0.001)


In [18]:
print('Mean F2: %.3f (%.3f)' % (f2_scores.mean(), f2_scores.std()))

Mean F2: 0.469 (0.002)


#### Credit card fraud detection is an anomaly detection where false negative are worse than false positive.
#### So we are gonna use the F2 Score as a metric !

#### Our baseline model has an F2 score of 0.47

In [ ]:
# Splitting the data into 60% training, 20 % Validation and 20 % Test
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
#tn, fp, fn, tp = confusion_matrix(y_val,predictions).ravel()

In [ ]:
#(tn, fp, fn, tp)

In [ ]:
# Tune hyperparameter with grid search

In [ ]:
## Export the baseline and the improved model

In [ ]:
import joblib
joblib.dump(model, 'model_name.pkl', compress=9)

In [ ]:
joblib.dump(model3, 'model_name.pkl', compress=9)

In [ ]:
# Export the testing dataset

In [ ]:
from numpy import savetxt
savetxt('X_test.csv', X_test, delimiter=',')
savetxt('y_test.csv', y_test, delimiter=',')